# Descrizione notebook
In questo notebook vengono puliti i dati e creati i file CSV. Si hanno le barzellette in file HTML e i rating degli utenti rispetto alle barezelle in file CSV, si puliscono i dati e si mettono dentro diversi CSV.

In [1]:
# import delle librerie necessarie
import re
import os
import codecs
import pandas as pd
import html2text
import numpy as np

# Creazione CVS delle barzellette

In [2]:
# creazione di una funzione che prende in input la posizione dei file HTML (barzellette)
# e restituisce una coppia <id_barzelletta, barzelletta>
def joke_extractor(file):
    # si apre il file richiesto con la codifica specificata
    data = codecs.open(file, 'r', encoding="cp1252")
    # si leggono i dati
    joke_html = data.read()
    # si prende il contenuto della pagina HTML, quindi la barzelletta
    joke = html2text.html2text(joke_html)
    # estrazione dell'id delle barzellette dal nome del file:
    # file.split(caratteri), sulla base dei caratteri inseriti restituisce un'array di 
    # due elementi: la parte di stringa che stava prima i caratteri specificati e 
    # la parte dopo
    joke_id = int(file.split('init')[1].split('.html')[0])
    return (joke_id, joke)

In [3]:
# si utilizza la funzione definita sopra per estrarre barzelletta ed id:
# si prendono tutti i nomi dei file presenti nel path specificato
files = os.listdir('data/raw/jokes')
jokes_list = []
# per ogni file si estrae la coppia <id_barzelletta, barzelletta> e si mette in una lista
for file in files:
    if file.endswith('.html'):
        jokes_list.append(joke_extractor('data/raw/jokes/'+ file))
# si crea un dataframe a partire dalla lista
df_jokes = pd.DataFrame(jokes_list, columns=['joke_id', 'joke'])
df_jokes.head(10)

,joke_id,joke
0,42,| \n\n \nTwo men are discussing the age old ...
1,19,| \n\n \nQ: If a person who speaks three lan...
2,88,\n \nA Czechoslovakian man felt his eyesigh...
3,93,"\n \nReaching the end of a job interview, t..."
4,52,| \n\n \nQ: What do Monica Lewinsky and Bob ...
5,5,| \n\n \nQ. What's O. J. Simpson's Internet ...
6,63,"| \n\n \nAn engineer, a physicist and a math..."
7,100,\n \nQ: Whats the difference between greeti...
8,27,| \n\n \nClinton returns from a vacation in ...
9,11,"| \n\n \nQ. What do a hurricane, a tornado, ..."


In [4]:
# si scrivono le barzellette e i relativi id su un CSV
df_jokes.to_csv('data/jokes.csv', index = None)

# Creazione CSV con i rating degli utenti (non normalizzati)

In [5]:
# funzione per sostituire i rating con valore 99 con NaN
def replace_99(ratings):
    # si prendono tutti i nomi delle colonne a partire dalla seconda, quindi 
    # si prendono i tutti gli identificatori delle barzellette
    joke_ids = ratings.columns[2:]
    # per ogni colonna si sostituiscono i 99 di quella colonna con NaN 
    for joke_id in joke_ids: 
        ratings[joke_id] = ratings[joke_id].replace([99],np.nan)
    return ratings

In [6]:
df_all_ratings = pd.DataFrame()

# si leggono i rating degli utenti
df1_ratings = pd.read_excel('data/raw/jester-data-1.xls', header = None) 
# df2_ratings = pd.read_excel('data/raw/jester-data-2.xls', header = None) 
# df3_ratings = pd.read_excel('data/raw/jester-data-3.xls', header = None) 

# df_all_ratings = pd.concat([df1_ratings, df2_ratings, df3_ratings], axis=0)
df_all_ratings = pd.concat([df1_ratings], axis=0)

# si assegna ad ogni utente un identificatore
# linespace(inizio, fine, elementi_totali_array): crea un array in base alle specifiche date
user_id = np.linspace(1, len(df_all_ratings), len(df_all_ratings))
# insert(posizione, nome_colonna, elementi): si inserisce una colonna
# nella posizione specificata con gli elementi specificati
df_all_ratings.insert(0, 'user_id', user_id)
# si rinominano un po' di colonne
df_all_ratings = df_all_ratings.rename({0: 'number_of_jokes_rated'}, axis='columns')
for i in range(1, 101):
    df_all_ratings = df_all_ratings.rename({i: 'joke_' + str(i)}, axis='columns')
df_all_ratings = df_all_ratings.astype({"user_id": int})
# si sostituiscono i 99 con i NaN
df_all_ratings = replace_99(df_all_ratings)
df_all_ratings.head(10)

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,1,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,3,49,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,4,48,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,5,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
5,6,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
6,7,47,NaN,NaN,NaN,NaN,8.59,-9.85,7.72,8.79,...,NaN,NaN,NaN,NaN,NaN,2.33,NaN,NaN,NaN,NaN
7,8,100,6.84,3.16,9.17,-6.21,-8.16,-1.70,9.27,1.41,...,7.23,-1.12,-0.10,-5.68,-3.16,-3.35,2.14,-0.05,1.31,0.00
8,9,100,-3.79,-3.54,-9.42,-6.89,-8.74,-0.29,-5.29,-8.93,...,4.37,-0.29,4.17,-0.29,-0.29,-0.29,-0.29,-0.29,-3.40,-4.95
9,10,72,3.01,5.15,5.15,3.01,6.41,5.15,8.93,2.52,...,NaN,4.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# si scrivono i rating degli utenti su un CSV
df_all_ratings.to_csv('data/no_normalized_ratings.csv', index = None)

# Creazione CSV con i rating degli utenti (normalizzati)
Si fa un pre-processamento dei dati che permette di normalizzare i dati. In particolare ai rating delle barzellette di ogni utente gli viene sottratto il rating medio dell'utente stesso, così da rendere gli utenti confrontabili.

In [8]:
# funzione per normalizzare i rating degli utenti:
# si sottrae ad ogni rating di una barzelletta il rating medio dell'utente che ha valutato quella barzelletta
def normalization(ratings):
    # shape resituisce una coppia <num_righe, num_colonna>, si prende il primo elemento della 
    # coppia, quindi numero di righe (utenti totali)
    total_users = ratings.shape[0]
    # iloc[i, 2:] della i-esima riga si prendono i valori dalla seconda colonna in poi e 
    # ad ogni valore si sottrae la media dei valori di quella riga (righe e colonne partono da 0)
    for i in range(total_users):
        ratings.iloc[i, 2:] = ratings.iloc[i, 2:] - np.mean(ratings.iloc[i, 2:])
    return ratings

In [9]:
# normalizzazione di tutti i rating con le funzioni sopra definite
normalized_ratings = normalization(df_all_ratings)
normalized_ratings.head(10)

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,1,74,-4.388108,12.221892,-6.228108,-4.728108,-4.088108,-5.068108,-6.418108,7.601892,...,6.251892,NaN,NaN,NaN,NaN,NaN,-2.198108,NaN,NaN,NaN
1,2,100,1.333700,-3.036300,3.613700,1.623700,-5.126300,-12.406300,-3.476300,-8.086300,...,0.073700,-7.696300,-3.036300,5.113700,-2.936300,-4.886300,0.313700,-2.406300,-7.066300,-1.676300
2,3,49,NaN,NaN,NaN,NaN,1.930612,2.170612,1.930612,2.170612,...,NaN,NaN,NaN,1.980612,NaN,NaN,NaN,NaN,NaN,NaN
3,4,48,NaN,5.691875,NaN,NaN,-0.858125,5.501875,-5.478125,3.551875,...,NaN,NaN,NaN,-2.128125,NaN,NaN,NaN,NaN,NaN,NaN
4,5,91,5.247363,1.357363,-7.422637,-8.642637,-1.892637,-1.652637,3.787363,1.357363,...,1.937363,2.327363,1.017363,1.937363,2.477363,-1.702637,-0.142637,3.297363,-1.452637,-1.652637
5,6,100,-1.272300,1.357700,5.337700,-3.602300,-2.192300,0.577700,-3.792300,4.027700,...,1.357700,-1.992300,4.217700,1.937700,2.717700,1.547700,4.947700,-4.182300,-0.152300,1.447700
6,7,47,NaN,NaN,NaN,NaN,4.068723,-14.371277,3.198723,4.268723,...,NaN,NaN,NaN,NaN,NaN,-2.191277,NaN,NaN,NaN,NaN
7,8,100,9.226900,5.546900,11.556900,-3.823100,-5.773100,0.686900,11.656900,3.796900,...,9.616900,1.266900,2.286900,-3.293100,-0.773100,-0.963100,4.526900,2.336900,3.696900,2.386900
8,9,100,-2.995600,-2.745600,-8.625600,-6.095600,-7.945600,0.504400,-4.495600,-8.135600,...,5.164400,0.504400,4.964400,0.504400,0.504400,0.504400,0.504400,0.504400,-2.605600,-4.155600
9,10,72,-1.557083,0.582917,0.582917,-1.557083,1.842917,0.582917,4.362917,-2.047083,...,NaN,-0.097083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# si scrivono i rating degli utenti su un CSV
normalized_ratings.to_csv('data/normalized_ratings.csv', index = None)

# Separazione degli utenti che hanno valutato tutte le barzellette con quelli che invece ne hanno valutate solo alcune
Si separano i rating noramlizzati in due parti: 
1. una parte contenente tutti gli utenti che hanno valutato tutte e 100 le barzellette;
2. una parte contenente tutti gli utenti che non hanno valutato tutte le barzellette.

In [11]:
# utenti che hanno valutato tutte e 100 le barzellette
complete_ratings = df_all_ratings[df_all_ratings['number_of_jokes_rated'] == 100]
complete_ratings.to_csv('data/complete_ratings_matrix.csv', index = None)